## MUMIA: PROCESAMIENTO DEL LENGUAJE NATURAL
### Caso grupal: Etiquetado morfosintáctico
### GRUPO 4 - EQUIPO 11 - PLN_11
### INTEGRANTES:

+ **Ignacio López Pérez**  
+ **Jose Angel de Bustos Pérez** 
+ **Marcos García Padrón**
+ **Jorge Valenzuela Jimenez**
+ **Javior Peralta Muñoz**


 ### Etiquetador morfosintáctico

In [1]:
%config IPCompleter.greedy=True
import re
import numpy as np
'''
   
   Funcion: Generar y popular los valores en las matrices de Observacion y Transicion
   Parametros:
             - lines: Lineas del corpus pasado en el fichero proporcionado
             - IN/OUT: typeTransictionDictionary: Estructura final para la Matriz de transición de probabilidades
             - IN/OUT: typeObservationDictionary: Estructura final para la matriz de observación de lemas
             
'''
def populateMatrix(lines, typeTransictionDictionary, typeObservationDictionary):
    previousPosTag = ''
    for line in lines:
        #Obviamos el nodo de estructra de metada de definción del documento (tag xml)
        metaDataRow = re.search("<doc |</doc", line.strip(), re.I)
        emptyLine = not line.strip()
        if (emptyLine):
            previousPosTag = '<s>'
        if not(metaDataRow):
            if not emptyLine:            
                token = line.split()[0].lower() #Nos quedamos con el lema
                posTag = line.split()[2] # Recogemos el posTAG en formato EAGLE
                if (posTag and token):
                    # Matriz de Observación
                    if not(posTag in typeObservationDictionary): # Si no está el POSTAG en la matriz de Observacion, lo metemos
                        typeObservationDictionary[posTag] = {}
                    if not(token in typeObservationDictionary[posTag]):# Si no está el lema en la matriz de Observacion, lo metemos
                        typeObservationDictionary[posTag][token] = 0
                    typeObservationDictionary[posTag][token] += 1 #Incrementamos las ocurrencias que aparece el termino para el POSTAG tratado
                     #Matriz de Transición
                    if not(posTag in typeTransictionDictionary):
                        typeTransictionDictionary[posTag] = {}
                    # Nos aseguramos que el tag t(i-1) está incluido en la matriz de transición- Contemplamos FP tambien
                    if not(previousPosTag in typeTransictionDictionary):
                        typeTransictionDictionary[previousPosTag] = {}
                    # incluimos la celda apropiada a la matriz
                    if not(posTag in typeTransictionDictionary[previousPosTag]):
                        typeTransictionDictionary[previousPosTag][posTag] = 0
                    # Incrementamos las ocurrencias de transición en la matriz
                    typeTransictionDictionary[previousPosTag][posTag] += 1
                previousPosTag = posTag # Actualización
'''
    Función: Generar probabilidades de la estructira pasada por parámetro
    inputdiccionary: MAtrizx con ocurrencias para calcular las proabialidaded
    return: Matriz con probabilidades ya calculadas
'''                
def calculateProbabilities(inputDictionary):
    outputDictionary = {}
    for keyDictionary in inputDictionary:
        outputDictionary[keyDictionary] = {} #Inicialización
        totalSum = sum(list(inputDictionary[keyDictionary].values())) #total de ocurrencias del termino tratado
        for key in inputDictionary[keyDictionary]:            
            outputDictionary[keyDictionary][key] = inputDictionary[keyDictionary][key] / totalSum # probabilidad
    return outputDictionary
            


In [2]:
#Leemos le ficheor proporcionado con el corpus asociado en formato EAGLE
file = open('./mia07_t3_ag_Corpus-tagged.txt' , 'r+')
lines = file.readlines()
file.close()
#Creamos las matrizces de Obdsrevacion y transicion para poder calcular las propabilidades 
typeTransictionDictionary = {}
typeObservationDictionary = {}
# Generammos las matrices con los valores para posteriormente calcular las probabilidades asociadas tanto
# de Observacion de lemas como de transición de etiquetas.
populateMatrix(lines, typeTransictionDictionary, typeObservationDictionary)
typeTransictionProbabilityDictionary = calculateProbabilities(typeTransictionDictionary)
typeObservationProbabilityDictionary = calculateProbabilities(typeObservationDictionary)   


In [3]:
#Mostramos la matriz
#TODO: Pintarla bonita si se puede NO SE ME OCURRE COMO LA VERDAD
typeObservationDictionary

{'NP00000': {'tristana': 1,
  'luis_buã±uel': 1,
  'benito_pã©rez_galdã³s': 1,
  'oscar': 1,
  'cole': 1,
  'centro_psiquiã¡trico': 2,
  'kathryn_railly': 1,
  'madeleine_stowe': 1,
  'jeffrey_goines': 1,
  'brad_pitt': 1,
  'monos': 1,
  'mr._bean': 1,
  'irma_gobb': 1,
  'denny_duquette': 1,
  'denny': 7,
  'burke': 1,
  'izzie': 2,
  'young': 4,
  'bob_egle': 1,
  'bovingdon_bug': 1,
  'biggs': 1},
 'VSIP3S0': {'es': 3},
 'DI0FS0': {'una': 4},
 'NCFS000': {'pelã\xadcula': 2,
  'novela': 1,
  'habla': 1,
  'relaciã³n': 1,
  'propagaciã³n': 1,
  'voz': 1,
  'novia': 1,
  'banalidad': 1,
  'historia': 1,
  'amistad': 1,
  'noticia': 1,
  'posibilidad': 1,
  'muerte': 1,
  'investigaciã³n': 1,
  'compaã±ã\xada': 1},
 'SPS00': {'de': 25,
  'en': 9,
  'a': 14,
  'sobre': 1,
  'con': 5,
  'a_pesar_de': 1,
  'por': 4,
  'sin': 1,
  'para': 3,
  'hasta': 1,
  'despuã©s_de': 1,
  'debido_a': 2,
  'luego_de': 1},
 'DA0MS0': {'el': 25},
 'NCMS000': {'director': 1,
  'nombre': 1,
  'viaje': 1,
 

In [4]:
#TODO Pintar la matriz de transición Bonita NO SE ME OCURRE COMO LA VERDAD
typeTransictionDictionary

{'NP00000': {'VSIP3S0': 1,
  'Fp': 4,
  'SPS00': 3,
  'VMIP3S0': 7,
  'Fc': 6,
  'Fpa': 2,
  'Fpt': 3,
  'VMIC1S0': 1,
  'CC': 2,
  'PP3CSD00': 1,
  'VAII1S0': 1,
  'Fe': 1},
 '<s>': {'NP00000': 3,
  'VAIP3S0': 1,
  'VSIS3S0': 1,
  'SPS00': 6,
  'VMIP3S0': 1,
  'CS': 1,
  'RG': 3},
 'VSIP3S0': {'DI0FS0': 1, 'RG': 1, 'VMP00SM': 1},
 'DI0FS0': {'NCFS000': 3, 'AQ0CS0': 1},
 'NCFS000': {'SPS00': 6,
  'RN': 1,
  'PR0CN000': 1,
  'AQ0CS0': 1,
  'Fe': 1,
  'PP3CNA00': 1,
  'P0000000': 1,
  'Fp': 1,
  'Fc': 1,
  'AQ0FS0': 1,
  'CC': 1},
 'SPS00': {'DA0MS0': 16,
  'DA0FS0': 7,
  'NP00000': 9,
  'NCFS000': 1,
  'Z': 1,
  'DP3CS0': 2,
  'CS': 4,
  'DI0MS0': 5,
  'VSN0000': 1,
  'DD0MS0': 3,
  'PR0CP000': 1,
  'DI0FP0': 1,
  'PR0CS000': 1,
  'DA0NS0': 2,
  'NCMP000': 3,
  'VMN0000': 4,
  'DP3CP0': 1,
  'DA0MP0': 2,
  'DI0FS0': 1,
  'NCMS000': 3},
 'DA0MS0': {'NCMS000': 15,
  'AQ0MS0': 1,
  'NP00000': 1,
  'PR0CN000': 1,
  'AQ0CS0': 1,
  'VMN0000': 3,
  'SPS00': 1,
  'DI0MS0': 1,
  'NCCS000': 1},
 

In [5]:
#TODO Pintar la matriz de Observacion Bonita NO SE ME OCURRE COMO LA VERDAD
typeObservationProbabilityDictionary

{'NP00000': {'tristana': 0.03125,
  'luis_buã±uel': 0.03125,
  'benito_pã©rez_galdã³s': 0.03125,
  'oscar': 0.03125,
  'cole': 0.03125,
  'centro_psiquiã¡trico': 0.0625,
  'kathryn_railly': 0.03125,
  'madeleine_stowe': 0.03125,
  'jeffrey_goines': 0.03125,
  'brad_pitt': 0.03125,
  'monos': 0.03125,
  'mr._bean': 0.03125,
  'irma_gobb': 0.03125,
  'denny_duquette': 0.03125,
  'denny': 0.21875,
  'burke': 0.03125,
  'izzie': 0.0625,
  'young': 0.125,
  'bob_egle': 0.03125,
  'bovingdon_bug': 0.03125,
  'biggs': 0.03125},
 'VSIP3S0': {'es': 1.0},
 'DI0FS0': {'una': 1.0},
 'NCFS000': {'pelã\xadcula': 0.125,
  'novela': 0.0625,
  'habla': 0.0625,
  'relaciã³n': 0.0625,
  'propagaciã³n': 0.0625,
  'voz': 0.0625,
  'novia': 0.0625,
  'banalidad': 0.0625,
  'historia': 0.0625,
  'amistad': 0.0625,
  'noticia': 0.0625,
  'posibilidad': 0.0625,
  'muerte': 0.0625,
  'investigaciã³n': 0.0625,
  'compaã±ã\xada': 0.0625},
 'SPS00': {'de': 0.36764705882352944,
  'en': 0.1323529411764706,
  'a': 0.

In [6]:
#TODO Pintar la matriz de probabilidades de transición Bonita.. NO SE ME OCURRE COMO LA VERDAD
typeTransictionProbabilityDictionary

{'NP00000': {'VSIP3S0': 0.03125,
  'Fp': 0.125,
  'SPS00': 0.09375,
  'VMIP3S0': 0.21875,
  'Fc': 0.1875,
  'Fpa': 0.0625,
  'Fpt': 0.09375,
  'VMIC1S0': 0.03125,
  'CC': 0.0625,
  'PP3CSD00': 0.03125,
  'VAII1S0': 0.03125,
  'Fe': 0.03125},
 '<s>': {'NP00000': 0.1875,
  'VAIP3S0': 0.0625,
  'VSIS3S0': 0.0625,
  'SPS00': 0.375,
  'VMIP3S0': 0.0625,
  'CS': 0.0625,
  'RG': 0.1875},
 'VSIP3S0': {'DI0FS0': 0.3333333333333333,
  'RG': 0.3333333333333333,
  'VMP00SM': 0.3333333333333333},
 'DI0FS0': {'NCFS000': 0.75, 'AQ0CS0': 0.25},
 'NCFS000': {'SPS00': 0.375,
  'RN': 0.0625,
  'PR0CN000': 0.0625,
  'AQ0CS0': 0.0625,
  'Fe': 0.0625,
  'PP3CNA00': 0.0625,
  'P0000000': 0.0625,
  'Fp': 0.0625,
  'Fc': 0.0625,
  'AQ0FS0': 0.0625,
  'CC': 0.0625},
 'SPS00': {'DA0MS0': 0.23529411764705882,
  'DA0FS0': 0.10294117647058823,
  'NP00000': 0.1323529411764706,
  'NCFS000': 0.014705882352941176,
  'Z': 0.014705882352941176,
  'DP3CS0': 0.029411764705882353,
  'CS': 0.058823529411764705,
  'DI0MS0': 0

In [7]:
path = ""
#Funciones de apoyo para el calculo de vitirbe
#----------------------------------------------
#Inicialización de la Matriz
#---------------------------------------------
def initializeMatrix(xLen, posTags):
    outputMatrix = {}
    for i in range(xLen + 1):
        outputMatrix[i] = {}
        for posTag in posTags:
            outputMatrix[i][posTag] = 0
    return outputMatrix
#----------------------------------------------------------------------------
#Devuelve la probilidad asociada a un token específico en una matriz concreta
#----------------------------------------------------------------------------
def getProbability(dictionary, basedOn, probabilityOf):
    if basedOn in dictionary:
        if probabilityOf in dictionary[basedOn]:
            return dictionary[basedOn][probabilityOf] #Probabilidad buscada 
    return 0
#--------------------------------------------------------------------------------------------------
#Limpieza de elementos espurios para asegurarnos y no tener errores que con tokenizer se eliminarian
#--------------------------------------------------------------------------------------------------
def sentenceCleanup(sentence):
    tokens = ['.', '/', '(', ')', ',']
    output = sentence.lower()
    for token in tokens:
        output = output.replace(token, ' ' + token + ' ')
    return output.split()
#-----------------------------------------------------------------------------------------
# ALGORITMO DE VITERBI
#
# sentence: Input a calcular el algoritmo de Viterbi
# observationProbabilities: Matriz de probabilidades de obdservacion
# transictionProbabilities: Matriz de probabilidades de Transición
#---------------------------------------------------------------------------------------

def calculateViterbiPath(sentence, observationProbabilities, transictionProbabilities):
    sentenceList = sentenceCleanup(sentence) #Aseguramos la elimación de elementos erróneos
    #print(sentenceList) 
    sentenceLen = len(sentenceList)
    posTagCount = len(transictionProbabilities)
    viterbiValues = initializeMatrix(sentenceLen + 1, transictionProbabilities)
    #recorremos cada morfema de la frase pasada por parametro
    path = ""
    for tokenIndex in range(sentenceLen + 1):
        if (tokenIndex == 0): #Elemento start 
            viterbiValues[tokenIndex]['<s>'] = 1
        else:
            token = sentenceList[tokenIndex - 1].lower() #morfema ques estamos tratando
            print(token)
            for posTagTo in transictionProbabilities:
                for posTagFrom in transictionProbabilities:
                    
                    # Calculamos las probabilidades tanto de trasición comom de observación para elaborar el camino
                    transictionProbability = getProbability(transictionProbabilities, posTagFrom, posTagTo)
                    observationProbability = getProbability(observationProbabilities, posTagTo, token)
                    newValue = viterbiValues[tokenIndex-1][posTagFrom] * transictionProbability * observationProbability
                    currentValue = viterbiValues[tokenIndex][posTagTo]
                    # Se nos pide mostrar TODAS LAS CELDAS de viterbi
                    #TODO ¿Lo dejamos? No sé... ensucia más que aclara...
                    '''
                    print('------------------------------------------------------------------------------------')
                    print('Observación de :' + str(token) + ' hacia ' + str(posTagTo) + '= ' + str(observationProbability))
                    print('Transicion desde: ' +  str(posTagFrom) + ' hasta ' + str(posTagTo) + str(transictionProbability))
                    print('Valor de Viterbi(' + str(tokenIndex-1) + ')' + '(' + str(posTagFrom) + ') = ' + str(newValue))
                    print('Valor de Viterbi(' + str(tokenIndex) +  ')' + '(' + str(posTagTo) + ') = ' + str(newValue))
                    print('------------------------------------------------------------------------------------')
                    '''                    
                    # Solo seleccionamos el maximo para la correcta implmentación del algoritmo 
                    # (el valor ya se multiplica con las probabilidades)
                    if (newValue > currentValue):
                        viterbiValues[tokenIndex][posTagTo] = newValue
                        #TODO: AQUI PODRIAMOS COGER EL MAXIMO DE LA COLUMNA DEL TOKEN PARA QUE EL ANALISIS FINAL 
                        # QUEDE BIEN. METE A LOS DOS POSTAGS EN LA MATRIZ DE VITERBI PORQUE SON INDICES DIFERENTES, ERGO AL
                        # COGER EL PATH FINAL METE LOS DOS.. HAY QUE SACAR EL POSTAG MAX np.amax(viterbiValues[tokenIndex])
                        # PARA QUE LA VISUALZIACION FINAL QUEDE BIEN... AUN ASÍ SE EXPLICA. VEMOS SI LO CAMBIAMOS.
                         # to demostrate only, and to choose the correct viterbi path -TODO-
                        print('from ' + posTagFrom +' to ' + posTagTo + ' in ' + token + ' NEW VALUE: ' + str(newValue))
                        path = path + " " + str(token) + "/" + str(posTagTo) 
    return path        

### Etiquetar morfosintácticamente una oración
* **Path de Viterbi**

In [8]:
etiqMS = calculateViterbiPath("Habla con el enfermo grave de trasplantes.", typeObservationProbabilityDictionary, typeTransictionProbabilityDictionary)

habla
from <s> to VMIP3S0 in habla NEW VALUE: 0.009259259259259259
con
from VMIP3S0 to SPS00 in con NEW VALUE: 0.0001765109335915436
el
from SPS00 to DA0MS0 in el NEW VALUE: 4.1531984374480846e-05
enfermo
from DA0MS0 to NCMS000 in enfermo NEW VALUE: 2.020474915515284e-06
from DA0MS0 to AQ0MS0 in enfermo NEW VALUE: 2.373256249970334e-07
grave
from NCMS000 to AQ0CS0 in grave NEW VALUE: 3.854642128169064e-08
de
from AQ0CS0 to SPS00 in de NEW VALUE: 2.5008491315975763e-09
trasplantes
from SPS00 to NCMP000 in trasplantes NEW VALUE: 2.7582894833796797e-11
.
from NCMP000 to Fp in . NEW VALUE: 3.4478618542245997e-12


### Etiquetado morfosintactico final

In [9]:
print(etiqMS)

 habla/VMIP3S0 con/SPS00 el/DA0MS0 enfermo/NCMS000 enfermo/AQ0MS0 grave/AQ0CS0 de/SPS00 trasplantes/NCMP000 ./Fp


**¿Es correcto el etiquetado morfosintáctico que has obtenido? Indica por qué.**
Vemos claramente  que el analizador ha funcionado correctamente asignando las mayores probabilidades a los términos correctamente y escogiendo el mejor path en aquellos casos de más de una transicion posible, escogiendo el maximo entre las mismas. 
Así podemos observar que para:
+ **HABLA** : lo etiqueta como Verbo General Indicativo Personal de Tercera persona del Singular
+ **CON** :  lo etiqueta perfectamente como una adposición - preposición (SP)
+ **EL** :  lo etiqueta concretamente como un determinante articulo MAsculino Singular
+ **ENFERMO**: En este caso, tenemos dos valores para el termino enfermo. Nombre y Adjetivo. Observamos que la probabilida de nombre es mayor que la de Adjetivo AQ 2.3 e-07 y NC 2.02 e-06, ergo lo está etiquetando tambien correctamente ademas observamos que la transicion siguiente es desde NCMS000 que es NOMBRE COMUN.
+ **GRAVE**: lo etiqueta como adjetivo cualitativo correctemente
+ **DE**: lo etiqueta correctamente como preposicion SP
+ **TRASPLANTES**: lo etiqueta correctamente como un nombre Comun.

En definitiva, **todos los morfemas de la oración los ha etiqueta correctemente**.

**▸Indica el resultado de etiquetar la oración «El enfermo grave habla de trasplantes.» utilizando el etiquetador morfosintáctico. ¿Es correcto el etiquetado morfosintáctico que has obtenido? Indica por qué.**

Ejecutamos nuestro analizador con la frase pedida

In [10]:
calculateViterbiPath("El enfermo grave de el que nadie tiene sospechas.", typeObservationProbabilityDictionary, typeTransictionProbabilityDictionary)

el
enfermo
grave
de
el
que
nadie
tiene
sospechas
.


''

Observamos que efectivamente el analisis no es correcto. Esto es debido a que en el corpus sumnistrado para el ejercicio no hay ninguna frase que empiece por un artículo singular masculino. (se ha chekeado igualmente manaulmente para asegurarnos) Esto implica que, como el corpus es incompleto, el etiquetado morfo sintáctico no se pueda llevar acabo.

**▸	¿Cuáles son las limitaciones del analizador morfosintáctico que has creado?**
* Precisamente como hemosm visto anteriormente el haber generado las matrices de observación y transición a partir de un corpus incompleto no pudiendose evaluar muchas variantes del lenguaje, por ejemplo vemos que las frases que comienzan con un articulo masculino singular, no los puede analizar.

In [11]:
calculateViterbiPath(" el corazon con quien entabla amistad.", typeObservationProbabilityDictionary, typeTransictionProbabilityDictionary)

el
corazon
con
quien
entabla
amistad
.


''

al igual que las que empiezan con un nombre comun

In [12]:
calculateViterbiPath("Corazon con quien entabla amistad.", typeObservationProbabilityDictionary, typeTransictionProbabilityDictionary)

corazon
con
quien
entabla
amistad
.


''

siendo pues un analizador bastante limitado.

**▸Qué posibles mejoras se podrían aplicar para mejorar el rendimiento del etiquetador morfosintáctico creado?**

+ Se podria crear reglas de exclusión en el lenguaje de tal manera que se minimizara el recorrido y todas las variantes que se tienen que calcular en las matrices. La reglas de exclusión sería combinaciones de estados que el propio lenguaje no puede aaprecer por no ser correcto gramaticalmente. De esta manera se chekearia si existe una transcición en la lista de exclusión, y con estoe no habria que evaluar ni calcular las probabilidaddes de ti-1 a ti puesto que no tendría sentido al no ser una combinacion correcta, ofreciendo de esta manera un mejor rendimiento a nivel computacional e igualmetne a nivel de almacenamiento de memoria.  
